In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = "https://www.youtube.com/@PW-Foundation/videos"
driver.get(url)

wait = WebDriverWait(driver, 10)
actions = ActionChains(driver)

for _ in range(3):
    actions.send_keys(Keys.PAGE_DOWN).perform()
    time.sleep(2)

wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="video-title-link"]')))

video_urls = []
thumbnails = []
titles = []
views = []
post_times = []

videos = driver.find_elements(By.XPATH, '//*[@id="video-title-link"]')[:5]
for i, video in enumerate(videos):
    try:
        title = video.get_attribute('title')
        titles.append(title)

        video_url = video.get_attribute('href')
        video_urls.append(video_url)

        thumbnail = driver.find_elements(By.XPATH, '//*[@id="thumbnail"]//img')[i]
        thumbnail_url = thumbnail.get_attribute('src')
        thumbnails.append(thumbnail_url)

        metadata_line = driver.find_elements(By.XPATH, '//*[@id="metadata-line"]/span')
        views.append(metadata_line[2 * i].text if len(metadata_line) > 2 * i else None)
        post_times.append(metadata_line[2 * i + 1].text if len(metadata_line) > 2 * i + 1 else None)
    
    except Exception as e:
        print(f"Error extracting data for video {i + 1}: {e}")
        titles.append(None)
        video_urls.append(None)
        thumbnails.append(None)
        views.append(None)
        post_times.append(None)

driver.quit()

data = {
    'Video URL': video_urls,
    'Thumbnail URL': thumbnails,
    'Title': titles,
    'Views': views,
    'Post Time': post_times
}

df = pd.DataFrame(data)
df.to_csv('youtube_videos.csv', index=False)

print(df)


                                     Video URL  \
0  https://www.youtube.com/watch?v=WwG1TatqoIc   
1  https://www.youtube.com/watch?v=ctI76G3ZC4k   
2  https://www.youtube.com/watch?v=EA0Hv-d5dv0   
3  https://www.youtube.com/watch?v=bOmZTd2u8rM   
4  https://www.youtube.com/watch?v=CMKnF8f6k4U   

                                       Thumbnail URL  \
0  https://i.ytimg.com/vi/WwG1TatqoIc/hqdefault.j...   
1  https://i.ytimg.com/vi/ctI76G3ZC4k/hqdefault.j...   
2  https://i.ytimg.com/vi/EA0Hv-d5dv0/hqdefault.j...   
3  https://i.ytimg.com/vi/bOmZTd2u8rM/hqdefault.j...   
4  https://i.ytimg.com/vi/CMKnF8f6k4U/hqdefault.j...   

                                               Title       Views     Post Time  
0  7 Powerful Tips to Cover Syllabus in Less Time...   64K views  18 hours ago  
1    Abh Hogi FINAL RACE💀🔥 || Class 9th FREE Batch⚡️   24K views     1 day ago  
2  Minerals and Energy Resources ONE SHOT | Full ...   12K views    3 days ago  
3  Consumer Rights in 28 Minutes || Mi